In [1]:
import config

from pyspark.sql import SparkSession
import pyspark.sql.functions as ps
from pyspark.sql import Window

In [2]:
spark = SparkSession.builder\
    .config('spark.driver.extraClassPath', '/home/vermichel/Downloads/postgresql-42.4.0.jar')\
    .getOrCreate()

spark_config = spark.read.format("jdbc")\
    .option("url", "jdbc:postgresql://localhost:5432/pagila") \
    .option("driver", "org.postgresql.Driver")\
    .option("user", "postgres")\
    .option("password", "password")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/25 15:46:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
category = spark_config.option("dbtable", "category").load()
film_category = spark_config.option("dbtable", "film_category").load()
film_actor = spark_config.option("dbtable", "film_actor").load()
actor = spark_config.option("dbtable", "actor").load()
film = spark_config.option("dbtable", "film").load()
rental = spark_config.option("dbtable", "rental").load()
inventory = spark_config.option("dbtable", "inventory").load()
payment = spark_config.option("dbtable", "payment").load()
address = spark_config.option("dbtable", "address").load()
city = spark_config.option("dbtable", "city").load()
customer = spark_config.option("dbtable", "customer").load()

In [4]:
# 1
result1 = category.join(film_category, category.category_id == film_category.category_id, "inner")
result1.groupBy("name")\
    .agg(ps.count("film_id").alias("films_count"))\
    .orderBy("name")\
    .show()

+-----------+-----------+
|       name|films_count|
+-----------+-----------+
|     Action|         64|
|  Animation|         66|
|   Children|         60|
|   Classics|         57|
|     Comedy|         58|
|Documentary|         68|
|      Drama|         62|
|     Family|         69|
|    Foreign|         73|
|      Games|         61|
|     Horror|         56|
|      Music|         51|
|        New|         63|
|     Sci-Fi|         61|
|     Sports|         74|
|     Travel|         57|
+-----------+-----------+



In [5]:
# 2
result2 = film_actor.join(film, film_actor.film_id == film.film_id, "inner")\
    .join(actor, film_actor.actor_id == actor.actor_id, "inner")
result2.groupBy("first_name", "last_name")\
    .agg(ps.count("rental_duration").alias("rentals"))\
    .orderBy("rentals", ascending=False)\
    .show(10)

+----------+-----------+-------+
|first_name|  last_name|rentals|
+----------+-----------+-------+
|     SUSAN|      DAVIS|     54|
|      GINA|  DEGENERES|     42|
|    WALTER|       TORN|     41|
|      MARY|     KEITEL|     40|
|   MATTHEW|     CARREY|     39|
|    SANDRA|     KILMER|     37|
|  SCARLETT|      DAMON|     36|
|       UMA|       WOOD|     35|
|   GROUCHO|      DUNST|     35|
|    ANGELA|WITHERSPOON|     35|
+----------+-----------+-------+
only showing top 10 rows



In [6]:
# 3 
result3 = payment.join(rental, payment.rental_id == rental.rental_id, "inner")\
    .join(inventory, rental.inventory_id == inventory.inventory_id, "inner")\
    .join(film_category, inventory.film_id == film_category.film_id, "inner")\
    .join(category, film_category.category_id == category.category_id)
result3.groupBy("name").agg(ps.sum("amount").alias("payed_money"))\
    .orderBy("payed_money", ascending=False).show(1)

+------+-----------+
|  name|payed_money|
+------+-----------+
|Sports|    5314.21|
+------+-----------+
only showing top 1 row



In [16]:
# 4
result4 = film.join(inventory, film.film_id == inventory.film_id, "left")\
    .where(inventory.film_id.isNull())
result4.select("title").show()

+--------------------+
|               title|
+--------------------+
|      CHOCOLATE DUCK|
|       BUTCH PANTHER|
|        VOLUME HOUSE|
|      ORDER BETRAYED|
|        TADPOLE PARK|
|    KILL BROTHERHOOD|
|FRANKENSTEIN STRA...|
|    CROSSING DIVORCE|
|    SUICIDES SILENCE|
|       CATCH AMISTAD|
|     PERDITION FARGO|
|       FLOATS GARDEN|
|           GUMP DATE|
|        WALLS ARTIST|
|  GLADIATOR WESTWARD|
|         HOCUS FRIDA|
|ARSENIC INDEPENDENCE|
|         MUPPET MILE|
|   FIREHOUSE VIETNAM|
|       ROOF CHAMPION|
+--------------------+
only showing top 20 rows



In [18]:
# 4
result4 = film.join(inventory, film.film_id == inventory.film_id, "left_semi")
result4.select("title").show()

+-------------------+
|              title|
+-------------------+
|   INSTINCT AIRPORT|
|    ISLAND EXORCIST|
|      KICK SAVANNAH|
|    SPLENDOR PATTON|
|    DOORS PRESIDENT|
|       HALL CASSIDY|
|       LUCKY FLYING|
|   NEWTON LABYRINTH|
|      ROCK INSTINCT|
|      SUBMARINE BED|
|       TORQUE BOUND|
|      APACHE DIVINE|
|        LEGEND JEDI|
|   BONNIE HOLOCAUST|
|     CHARADE DUFFEL|
|DRAGONFLY STRANGERS|
|         IGBY MAKER|
|        MINE TITANS|
|         SLING LUKE|
|   BEHAVIOR RUNAWAY|
+-------------------+
only showing top 20 rows



In [92]:
# 5
data = actor.join(film_actor, actor.actor_id == film_actor.actor_id, 'inner')\
    .join(film_category, film_actor.film_id == film_category.film_id, 'inner')\
    .join(category, film_category.category_id == category.category_id, 'inner')\
    .where(category.name == "Children")
data = data.groupBy("first_name", "last_name")\
    .agg(ps.count("name")\
    .alias("top")).orderBy("top", ascending=False)
w = Window.orderBy(data.top.desc())
data = data.withColumn("newtop", ps.dense_rank().over(w))
data.where(data.newtop <= 3).show()

22/08/25 17:08:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/25 17:08:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/25 17:08:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/25 17:08:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/25 17:08:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/25 17:08:09 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/08/25 1

In [89]:
# 6
result6 = address.join(city, address.city_id == city.city_id, "inner")\
    .join(customer, address.address_id == customer.address_id, "inner")\
    .groupBy("city")\
    .agg(ps.sum(ps.when(customer.active == 1, 1).otherwise(0)).alias("active"),
         ps.sum(ps.when(customer.active != 1, 1).otherwise(0)).alias("inactive"))\
    .orderBy("inactive", ascending=False).show()

+------------------+------+--------+
|              city|active|inactive|
+------------------+------+--------+
|         Pingxiang|     0|       1|
|       Szkesfehrvr|     0|       1|
|  Charlotte Amalie|     0|       1|
|         Najafabad|     0|       1|
|           Wroclaw|     0|       1|
|            Ktahya|     0|       1|
|           Bat Yam|     0|       1|
|   Southend-on-Sea|     0|       1|
|            Amroha|     0|       1|
|            Kamyin|     0|       1|
|          Xiangfan|     0|       1|
|            Daxian|     0|       1|
|          Uluberia|     0|       1|
|     Coatzacoalcos|     0|       1|
|        Kumbakonam|     0|       1|
|A Corua (La Corua)|     1|       0|
|          Fengshan|     1|       0|
|          Chisinau|     1|       0|
|           Udaipur|     1|       0|
|              Linz|     1|       0|
+------------------+------+--------+
only showing top 20 rows



In [86]:
# 7
result7 = category.join(film_category, category.category_id == film_category.category_id)\
    .join(inventory, film_category.film_id == inventory.film_id)\
    .join(rental, inventory.inventory_id == rental.inventory_id)\
    .join(customer, rental.customer_id == customer.customer_id)\
    .join(address, customer.address_id == address.address_id)\
    .join(city, address.city_id == city.city_id)\
    .where(city.city.like('A%') | city.city.like('%-%'))
result7.groupBy("city", "name")\
    .agg(ps.sum(((ps.unix_timestamp(result7.return_date) - ps.unix_timestamp(result7.rental_date))/3600))
         .alias("hours"))\
    .orderBy("hours", ascending=False)\
    .select("name", "city", "hours").show(1)

+------+----------+------------------+
|  name|      city|             hours|
+------+----------+------------------+
|Action|Avellaneda|1109.8666666666666|
+------+----------+------------------+
only showing top 1 row

